<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Dash - Create an interactive plot
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Dash/Dash_Create_Datatable_With_Dropdown.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Dash+-+Create+Datatable+With+Dropdown:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #dash #dashboard #plotly #naas #asset #analytics #dropdown #callback #bootstrap #snippet

**Author:** [Zihui Ouyang](https://www.linkedin.com/in/zihui-ouyang-539626227/)

**Description:** This notebook creates an interactive plot using Dash app infrastructure.

**References:**
- https://stackoverflow.com/questions/70886359/dash-python-making-subplots-when-multiple-parameters-are-selected
- https://dash-example-index.herokuapp.com/line-charts

## Input

### Import libraries

In [1]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

To run the example below, one needs to put the data int he folder where this notebook is located. Here is the download link: https://ourworldindata.org/grapher/primary-energy-fossil-nuclear-renewables?facet=none

### Setup Variables
- `DASH_PORT`: specify a port number for Dash

In [3]:
DASH_PORT = 8050

## Model

### Initialize Dash app

In [ ]:
app = dash.Dash(
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

# app = dash.Dash() if you are not in Naas

### Data and a list that helps to create subplots

In [2]:
df = pd.read_csv("primary-energy-fossil-nuclear-renewables.csv")

PS = ["Nuclear (% direct primary energy)","Renewables (% direct primary energy)",
                    "Fossil fuels (% direct primary energy)"]

FileNotFoundError: [Errno 2] No such file or directory: 'primary-energy-fossil-nuclear-renewables.csv'

### Dash App

#### Create dropdown, table and layout

In [ ]:
app = Dash(__name__)
app.layout = html.Div(
    [
        html.H4("Direct primary energy consumption from 3 sources"),
        html.P("Select countries"),
        dcc.Dropdown(
            id="countries",
            options=df.Entity.unique(),
            value=["World"],
            multi = True
        ),
        *[dcc.Graph(id=p, figure={}, style={'display': 'none'}) for p in PS]
    ]
)

@callback(
    [Output(p, 'figure') for p in PS],
    [Output(p, 'style') for p in PS],
    Input('countries', 'value'),
)

def update_graph(country):
    dff = df.Entity.isin(country)
    figures = [{} for _ in range(len(PS))]
    styles = [{'display': 'none'} for _ in range(len(PS))]
    for i in range(3):
        figures[i] = px.line(df[dff], x='Year', y=PS[i], color = "Entity")
        styles[i]['display'] = 'block'    
    return *figures, *styles

## Output

### Generate URL and show logs

In [7]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")